<a href="https://colab.research.google.com/github/cildiroyku/TransferLearningforAnimalSounds/blob/main/DLProjectSetupandTemplateforExperimentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up to clone the repo from GitHub, get dependencies, model weights and connect to GPU

No need to modify this section. If GPU is not available, model will run on CPU.


In [ ]:
!git clone https://github.com/cildiroyku/TransferLearningforAnimalSounds.git
%cd TransferLearningforAnimalSounds
!ls -lh yamnet.pth

Cloning into 'TransferLearningforAnimalSounds'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 26 (delta 3), reused 13 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (26/26), 13.33 MiB | 18.47 MiB/s, done.
Resolving deltas: 100% (3/3), done.
/content/TransferLearningforAnimalSounds/TransferLearningforAnimalSounds
-rw-r--r-- 1 root root 15M Nov  4 23:44 yamnet.pth


In [ ]:
import yaml
cfg = yaml.safe_load(open("configs/base.yaml"))
print(cfg["data"]["sample_rate"])

16000


In [ ]:
!python test_config.py

Config loaded successfully!
Project name: transfer_learning_for_animal_sounds
Sample rate: 16000
Device preference: cuda

Verified folder: data/gtzan/
Verified folder: data/esc50/
Verified folder: outputs/embeddings/

GPU is available: NVIDIA A100-SXM4-40GB
